# 2022-06-30. A method to choose ensamble results


This Notebook has two goals:
- To get familiar with fastkaggle library, which give us the ability to work locally or on Kaggle. When working locally...

### OJO, No he hecho nada aquí

In [1]:
from fastai.vision.all import *
from fastcore.parallel import *

import timm
import gc

In [2]:
path = Path('paddy-disease-classification')

train_path = path / 'train_images'
train_files = get_image_files(train_path)

test_path = path/'test_images'
test_files = get_image_files(test_path).sorted()

In [3]:
train_df = pd.read_csv(path / 'train.csv')
train_df.sample(10)

,image_id,label,variety,age
4503,108908.jpg,dead_heart,ADT45,70
2371,103590.jpg,blast,KarnatakaPonni,75
558,100964.jpg,bacterial_leaf_streak,ADT45,68
278,101761.jpg,bacterial_leaf_blight,ADT45,72
6499,106452.jpg,hispa,ADT45,68
8767,105394.jpg,normal,ADT45,70
4038,106783.jpg,dead_heart,ADT45,67
5741,101237.jpg,downy_mildew,KarnatakaPonni,50
3720,106952.jpg,brown_spot,ADT45,72
2248,101063.jpg,blast,KarnatakaPonni,75


In [139]:
train_df.label.value_counts()

normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: label, dtype: int64

In [4]:
sample_submission = pd.read_csv(path/'sample_submission.csv')

## Exploring some Kaggle results

Estas predicciones salieron de copiar el Notebook de Jeremy. Pero en lugar de hacer el promedio, me suena que el promedio da una loquera.
https://www.kaggle.com/code/fmussari/scaling-up-road-to-the-top-part-3/edit

In [87]:
jeremy_df = pd.read_csv('2022-06-30_Experiment from Jeremy Kaggle Netebook Part 3.csv')
jeremy_df

,pred1,pred2,tta1,tta2
0,hispa,hispa,hispa,hispa
1,normal,normal,normal,normal
2,blast,blast,blast,blast
3,blast,blast,blast,blast
4,blast,blast,blast,blast
...,...,...,...,...
3464,dead_heart,dead_heart,dead_heart,dead_heart
3465,hispa,hispa,hispa,hispa
3466,normal,normal,normal,normal
3467,bacterial_leaf_streak,bacterial_leaf_streak,bacterial_leaf_streak,bacterial_leaf_streak


In [89]:
final = pd.DataFrame()
final.index = jeremy_df.index
final['results'] = None

In [90]:
jeremy_df['unique'] = jeremy_df.nunique(axis=1)
jeremy_df.sample(10)

,pred1,pred2,tta1,tta2,unique
3027,blast,blast,blast,blast,1
3196,dead_heart,dead_heart,dead_heart,dead_heart,1
2854,bacterial_leaf_blight,bacterial_leaf_blight,bacterial_leaf_blight,bacterial_leaf_blight,1
2166,normal,normal,normal,normal,1
1835,dead_heart,dead_heart,dead_heart,dead_heart,1
2994,brown_spot,tungro,tungro,tungro,2
2992,hispa,hispa,hispa,hispa,1
644,tungro,tungro,tungro,tungro,1
2529,bacterial_leaf_blight,bacterial_leaf_blight,bacterial_leaf_blight,bacterial_leaf_blight,1
929,tungro,tungro,tungro,tungro,1


In [91]:
jeremy_df[jeremy_df.unique == 2]

,pred1,pred2,tta1,tta2,unique
106,blast,tungro,blast,tungro,2
180,bacterial_leaf_streak,bacterial_leaf_streak,bacterial_leaf_streak,brown_spot,2
191,normal,hispa,normal,hispa,2
420,brown_spot,downy_mildew,brown_spot,brown_spot,2
422,bacterial_panicle_blight,normal,bacterial_panicle_blight,normal,2
...,...,...,...,...,...
3304,hispa,normal,normal,hispa,2
3329,normal,hispa,normal,normal,2
3343,downy_mildew,blast,blast,blast,2
3373,normal,blast,normal,blast,2


In [92]:
jeremy_df[jeremy_df.unique == 3]

,pred1,pred2,tta1,tta2,unique
655,blast,brown_spot,downy_mildew,downy_mildew,3
2266,brown_spot,dead_heart,bacterial_leaf_blight,brown_spot,3
2676,hispa,blast,bacterial_leaf_blight,blast,3
2855,hispa,tungro,blast,tungro,3


In [93]:
values_unisono = jeremy_df[jeremy_df.unique == 1].pred1
values_unisono

0                       hispa
1                      normal
2                       blast
3                       blast
4                       blast
                ...          
3464               dead_heart
3465                    hispa
3466                   normal
3467    bacterial_leaf_streak
3468               dead_heart
Name: pred1, Length: 3398, dtype: object

In [123]:
final.loc[values_unisono.index , 'results'] = values_unisono

In [125]:
sum(final.results.isnull())

71

In [131]:
values_impar3 = jeremy_df[jeremy_df.unique == 3].mode(axis=1)
values_impar3[0]

655     downy_mildew
2266      brown_spot
2676           blast
2855          tungro
Name: 0, dtype: object

In [132]:
final.loc[values_impar3[0].index , 'results'] = values_impar3[0]

In [133]:
sum(final.results.isnull())

67

Cuando hay solo 2 categoría, pero gana una 3 a 1

In [135]:
values_impar2 = jeremy_df[jeremy_df.unique == 2].mode(axis=1)
values_impar2_desempate = values_impar2[values_impar2[1].isna()]
values_impar2_desempate[0]

180     bacterial_leaf_streak
420                brown_spot
565                     blast
634                     blast
711                brown_spot
1052                    blast
1080                   tungro
1121                   normal
1317                    blast
1358                    hispa
1403                    hispa
1445                    hispa
1513                    blast
1516             downy_mildew
1539                   normal
1676                    hispa
1791                   tungro
1794                    blast
1933                   normal
2023               brown_spot
2177               brown_spot
2233                   tungro
2335                   normal
2448    bacterial_leaf_streak
2537                   normal
2550                    hispa
2636                   normal
2994                   tungro
2996                   normal
3038                    hispa
3102               brown_spot
3131                    hispa
3181                    hispa
3267      

In [136]:
final.loc[values_impar2_desempate[0].index , 'results'] = values_impar2_desempate[0]

In [137]:
sum(final.results.isnull())

30

In [174]:
values_impar2_empate = values_impar2[~values_impar2[1].isna()]

values_impar2_empate = values_impar2_empate.copy()

In [175]:
list(zip(values_impar2_empate[0], values_impar2_empate[1]))
values_impar2_empate['tuple'] = list(zip(values_impar2_empate[0], values_impar2_empate[1]))
values_impar2_empate

,0,1,tuple
106,blast,tungro,"(blast, tungro)"
191,hispa,normal,"(hispa, normal)"
422,bacterial_panicle_blight,normal,"(bacterial_panicle_blight, normal)"
506,blast,downy_mildew,"(blast, downy_mildew)"
518,brown_spot,tungro,"(brown_spot, tungro)"
553,blast,tungro,"(blast, tungro)"
660,bacterial_leaf_streak,blast,"(bacterial_leaf_streak, blast)"
685,blast,tungro,"(blast, tungro)"
887,brown_spot,tungro,"(brown_spot, tungro)"
927,hispa,normal,"(hispa, normal)"


In [176]:
values_impar2_empate.tuple.value_counts()

(hispa, normal)                       9
(blast, tungro)                       5
(blast, normal)                       3
(brown_spot, tungro)                  2
(hispa, tungro)                       2
(blast, brown_spot)                   2
(bacterial_panicle_blight, normal)    1
(blast, downy_mildew)                 1
(bacterial_leaf_streak, blast)        1
(downy_mildew, tungro)                1
(bacterial_leaf_blight, blast)        1
(bacterial_leaf_streak, tungro)       1
(bacterial_leaf_blight, hispa)        1
Name: tuple, dtype: int64

In [177]:
# From resnet34 16 epochs lr=0.005 confussion matrix
subs = {
    ('hispa', 'normal'): 'hispa',
    ('blast', 'tungro'): 'tungro',
    ('blast', 'normal'): 'blast',
    ('brown_spot', 'tungro'): 'tungro',
    ('hispa', 'tungro'): 'hispa',
    ('blast', 'brown_spot'): 'brown_spot',
    ('bacterial_panicle_blight', 'normal'): 'bacterial_panicle_blight',
    ('blast', 'downy_mildew'): 'downy_mildew',
    ('bacterial_leaf_streak', 'blast'): 'blast',
    ('downy_mildew', 'tungro'): 'tungro',
    ('bacterial_leaf_blight', 'blast'): 'blast',
    ('bacterial_leaf_streak', 'tungro'): 'tungro',
    ('bacterial_leaf_blight', 'hispa'): 'hispa'
    
}

In [186]:
values_impar2_empate.tuple.map(subs)

106                       tungro
191                        hispa
422     bacterial_panicle_blight
506                 downy_mildew
518                       tungro
553                       tungro
660                        blast
685                       tungro
887                       tungro
927                        hispa
1433                       hispa
1920                      tungro
1922                       hispa
1940                       hispa
2040                       blast
2093                       blast
2170                       hispa
2384                       hispa
2585                      tungro
2735                      tungro
2797                       hispa
2862                      tungro
2891                       hispa
2928                       hispa
2963                  brown_spot
3032                       hispa
3110                       blast
3256                  brown_spot
3304                       hispa
3373                       blast
Name: tupl

In [185]:
all(final[final.results.isnull()].index == values_impar2_empate.index)

True

In [188]:
final.loc[values_impar2_empate.index , 'results'] = values_impar2_empate.tuple.map(subs)

In [193]:
final.results.isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
3464    False
3465    False
3466    False
3467    False
3468    False
Name: results, Length: 3469, dtype: bool

In [196]:
sample_submission.label = final.results

In [197]:
sample_submission

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [198]:
sample_submission.to_csv("manual_tweking_to_jeremy_part3.csv", index=False)

In [199]:
!ls

'2022-06-28. Night Experiments - Scaling resnet34 .ipynb'
'2022-06-30. Pipeline with Progressive Resizing.ipynb'
'2022-06-30_Experiment from Jeremy Kaggle Netebook Part 3.csv'
'Afterburning Overclock Report.pbix'
'Experiment that Hanged.ipynb'
'Submissions & Experiments - Progressive Resizing.ipynb'
'Submissions & Experiments.ipynb'
 Walkthru12.ipynb
 Walkthru13.ipynb
 class_blast_vs_no-blast.pkl
 class_normal_vs_sick.pkl
 keys
 manual_tweking_to_jeremy_part3.csv
 model220628-224-resnet34-16epochs-lr005.pkl
 model220628-360-resnet34-16epochs-lr005.pkl
 model220628-480-resnet34-16epochs-lr01.pkl
 models
 paddy-disease-classification
 paddy-disease-classification.zip
 results-file.csv
 subm-224-resnet34-12epochs-lr01_tta.csv
 subm-224-resnet34-8epochs.csv
 subm224-resnet34-16epochs-lr005_tta.csv
 subm224-resnet34-16epochs-lr005_tta_normal_help.csv
 subm_2_tta.csv
 subm_3_tta.csv


In [201]:
sample_submission.label = jeremy_df.pred1
sample_submission

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [202]:
sample_submission.to_csv("pred1_jeremy_part3.csv", index=False)

In [203]:
sample_submission.label = jeremy_df.tta1
sample_submission

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [204]:
sample_submission.to_csv("tta1_jeremy_part3.csv", index=False)

In [205]:
sample_submission.label = jeremy_df.pred2
sample_submission.to_csv("pred2_jeremy_part3.csv", index=False)

In [206]:
sample_submission.label = jeremy_df.tta2
sample_submission.to_csv("tta2_jeremy_part3.csv", index=False)

In [209]:
jeremy_df[sample_submission.image_id == '200916.jpg']

,pred1,pred2,tta1,tta2,unique
915,brown_spot,brown_spot,brown_spot,brown_spot,1


In [210]:
jeremy_df[sample_submission.image_id == '200533.jpg']

,pred1,pred2,tta1,tta2,unique
532,brown_spot,brown_spot,brown_spot,brown_spot,1


In [212]:
jeremy_df.tta1.value_counts()

normal                      586
blast                       578
hispa                       540
dead_heart                  479
tungro                      362
brown_spot                  321
downy_mildew                192
bacterial_leaf_blight       168
bacterial_leaf_streak       130
bacterial_panicle_blight    113
Name: tta1, dtype: int64